In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_groq import ChatGroq
_ = load_dotenv(find_dotenv())
llm = ChatGroq(
    model_name="llama3-8b-8192",  # Options: llama3-8b-8192, llama3-70b-8192, mixtral-8x7b-32768
    groq_api_key=os.getenv("GROQ_API_KEY")
)


## Load private document

In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage


documents = SimpleDirectoryReader("data").load_data()
documents

[Document(id_='28e5e8fa-8d41-48c1-9286-b21003cafe95', embedding=None, metadata={'file_path': 'c:\\Users\\msshe\\Documents\\Projects\\LlamaIndexTutorials\\data\\be-good.txt', 'file_name': 'be-good.txt', 'file_type': 'text/plain', 'file_size': 16710, 'creation_date': '2024-10-05', 'last_modified_date': '2025-06-24'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Be good\n\nApril 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the\nphrase that became our motto: Make something people want.  We\'ve\nlearned a lot since then, but if I were choosing now that\'s still

## Create vector database

In [3]:
documents

[Document(id_='28e5e8fa-8d41-48c1-9286-b21003cafe95', embedding=None, metadata={'file_path': 'c:\\Users\\msshe\\Documents\\Projects\\LlamaIndexTutorials\\data\\be-good.txt', 'file_name': 'be-good.txt', 'file_type': 'text/plain', 'file_size': 16710, 'creation_date': '2024-10-05', 'last_modified_date': '2025-06-24'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Be good\n\nApril 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the\nphrase that became our motto: Make something people want.  We\'ve\nlearned a lot since then, but if I were choosing now that\'s still

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# 1. Load your documents
documents = SimpleDirectoryReader("data").load_data()

# 2. Use Hugging Face model for embeddings
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")



c:\Users\msshe\Documents\Projects\Demo-Langchain\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from llama_index.llms.langchain import LangChainLLM # type: ignore

# 3. Create the index using the non-OpenAI embedding model
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)


llm = ChatGroq(
    model_name="llama3-8b-8192",  # Options: llama3-8b-8192, llama3-70b-8192, mixtral-8x7b-32768
    groq_api_key=os.getenv("GROQ_API_KEY")
)

llm = LangChainLLM(llm=llm)

# 4. Query
query_engine = index.as_query_engine(llm = llm)
response = query_engine.query("What is this document about?")
print(response)

This document discusses the importance of being good in a business context, especially for startups. It argues that being good is not just a moral imperative but also a practical strategy for success. The author suggests that being good can improve morale, attract help from others, and enable decisive decision-making. The document also explores the idea that benevolence can be a valuable asset for companies, even those that are not traditionally associated with altruism, such as Google.


In [7]:
# index = VectorStoreIndex.from_documents(documents)

## Ask questions to private document

In [10]:
query_engine = index.as_query_engine(llm = llm)
response = query_engine.query("summarize the article in 100 words")
print(response)

The article discusses the importance of being good in business. It argues that successful startups often behave like nonprofits, prioritizing the needs of their users rather than seeking to make a profit. The author suggests that this approach can be lucrative, as users are more likely to rally around and support a company that is genuinely trying to make a positive impact. The article also highlights the benefits of being good, including attracting idealistic and talented employees, and serving as a compass for decision-making.


## See under the hood

In [9]:
# import logging
# import sys

# logging.basicConfig(
#     stream=sys.stdout, 
#     level=logging.DEBUG
# )

# logging.getLogger().addHandler(
#     logging.StreamHandler(stream=sys.stdout)
# )

## Save the vector database

In [11]:
import os.path

# check if storage already exists
if not os.path.exists("./storage"):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents, llm=llm)
    # store it for later
    index.storage_context.persist()
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir="./storage")
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine(llm=llm)




Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage\index_store.json.


ValueError: 
******
Could not load OpenAI embedding model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

Consider using embed_model='local'.
Visit our documentation for more embedding options: https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#modules
******

In [ ]:
response = query_engine.query("According to the author, what is good?")

print(response)

According to the author, being good is not just about having good intentions or claiming to be a good person. It is about being genuinely helpful, creating value for others, and having a positive impact on people's lives.


## Customization options
* parse into smaller chunks
* use a different vector store
* retrieve more context when I query
* use a different LLM
* use a different response mode
* stream the response back

## Use cases
* QA
* Chatbot
* Agent
* Structured Data Extraction
* Multimodal

## Optimizing
* Advanced Retrieval Strategies
* Evaluation
* Building performant RAG applications for production

## Other
* LlamaPacks and Create-llama = LangChain templates
* Very recent, still in beta
* Very interesting: create-llama allows you to create a Vercel app!
* Very interesting: open-source end-to-end project (SEC Insights)
    * llamaindex + react/nextjs (vercel) + fastAPI + render + AWS
    * environment setup: localStack + docker
    * monitoring: sentry
    * load testing: loader.io
    * [web](https://www.secinsights.ai/)
    * [code](https://github.com/run-llama/sec-insights)